## 재무 제표 open api로 받아오기

https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019020

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("DART_API_KEY")

## 1. 기업 코드 가져오기
- 사용자 질문 :  A기업 재무제표 어때?
- A기업의 이름으로 기업 코드를 찾아서, 그 기업코드로 재무제표 가져와야 함.

In [3]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET

url = "https://opendart.fss.or.kr/api/corpCode.xml"
params = {"crtfc_key": API_KEY}

response = requests.get(url, params=params)

if response.status_code == 200:
    # ZIP 파일 메모리에서 열기
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # ZIP 안에 XML 파일명 확인
        xml_filename = z.namelist()[0]

        # XML 파일 읽기
        with z.open(xml_filename) as f:
            tree = ET.parse(f)
            root = tree.getroot()

            # 기업 정보 리스트 저장용
            corp_list = []

            for child in root.findall("list"):
                corp_code = child.findtext("corp_code")
                corp_name = child.findtext("corp_name")
                stock_code = child.findtext("stock_code")
                corp_eng_name = child.findtext("corp_eng_name")
                modify_date = child.findtext("modify_date")

                corp_list.append({
                    "corp_code": corp_code,
                    "corp_name": corp_name,
                    "stock_code": stock_code,
                    "corp_eng_name": corp_eng_name,
                    "modify_date": modify_date,
                })

            print(f"총 기업 수: {len(corp_list)}")
            print("예시:", corp_list[:5])
else:
    print("API 요청 실패", response.status_code)


총 기업 수: 112846
예시: [{'corp_code': '00434003', 'corp_name': '다코', 'stock_code': ' ', 'corp_eng_name': 'Daco corporation', 'modify_date': '20170630'}, {'corp_code': '00430964', 'corp_name': '굿앤엘에스', 'stock_code': ' ', 'corp_eng_name': 'Good & LS Co.,Ltd.', 'modify_date': '20170630'}, {'corp_code': '00388953', 'corp_name': '크레디피아제이십오차유동화전문회사', 'stock_code': ' ', 'corp_eng_name': 'Credipia 25th Asset Securitization Specialty L.L.C', 'modify_date': '20170630'}, {'corp_code': '00179984', 'corp_name': '연방건설산업', 'stock_code': ' ', 'corp_eng_name': 'youn bao', 'modify_date': '20170630'}, {'corp_code': '00420143', 'corp_name': '브룩스피알아이오토메이션잉크', 'stock_code': ' ', 'corp_eng_name': 'BROOKS-PRI Automation, Inc.', 'modify_date': '20170630'}]


In [5]:
# corp_list 저장해두기 (json)

import json
with open('corp_list.json', 'w', encoding='utf-8') as f:
    json.dump(corp_list, f, ensure_ascii=False, indent=2)


In [6]:
# 저장된 corp_list 불러오기

with open('corp_list.json', encoding='utf-8') as f:
    corp_list = json.load(f)


## 1-1 회사명으로 corp_list에서 corp_code 찾기

In [7]:
def find_corp_code(company_name):
    # 간단 예: 정확 일치 찾기
    for corp in corp_list:
        if corp["corp_name"] == company_name:
            return corp["corp_code"]
    return None

corp_code = find_corp_code('삼성전자')
print(corp_code)

00126380


## 2. 특정 기업 재무 제표 open api로 받아오기

https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019020

In [11]:
import requests

corp_code = corp_code    # 위에서 찾은 corp_code
bsns_year = "2023"          # 조회 연도
reprt_code = "11011"        # 사업보고서
fs_div = "OFS"              # 개별재무제표 (연결은 "CFS")

url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

params = {
    "crtfc_key": API_KEY,
    "corp_code": corp_code,
    "bsns_year": bsns_year,
    "reprt_code": reprt_code,
    "fs_div": fs_div,
}

response = requests.get(url, params=params)
data = response.json()

data_list = []

if data["status"] == "000":
    # 재무제표 항목 리스트
    for item in data["list"]:
        name = item["account_nm"]
        curr = item["thstrm_amount"]
        prev = item.get("frmtrm_amount", "-")
        currency = item["currency"]
        data_list.append(f"{name} : {curr} (당기), {prev} (전기), 통화: {currency}")
else:
    print("API 오류:", data.get("message"))

In [12]:
data_list

['자산총계 : 296857289000000 (당기), 260083750000000 (전기), 통화: KRW',
 '유동자산 : 68548442000000 (당기), 59062658000000 (전기), 통화: KRW',
 '미수금 : 1910054000000 (당기), 2925006000000 (전기), 통화: KRW',
 '선급비용 : 1349755000000 (당기), 1047900000000 (전기), 통화: KRW',
 '현금및현금성자산 : 6061451000000 (당기), 3921593000000 (전기), 통화: KRW',
 '매출채권 : 27363016000000 (당기), 20503223000000 (전기), 통화: KRW',
 '재고자산 : 29338151000000 (당기), 27990007000000 (전기), 통화: KRW',
 '기타유동자산 : 2475944000000 (당기), 2674792000000 (전기), 통화: KRW',
 '단기금융상품 : 50071000000 (당기), 137000000 (전기), 통화: KRW',
 '비유동자산 : 228308847000000 (당기), 201021092000000 (전기), 통화: KRW',
 '이연법인세자산 : 9931358000000 (당기), 2142512000000 (전기), 통화: KRW',
 '무형자산 : 10440211000000 (당기), 8561424000000 (전기), 통화: KRW',
 '종속기업, 관계기업 및 공동기업 투자 : 57392438000000 (당기), 57397249000000 (전기), 통화: KRW',
 '당기손익-공정가치금융자산 : 1000000 (당기), 283000000 (전기), 통화: KRW',
 '기타포괄손익-공정가치 측정 비유동금융자산 : 1854503000000 (당기), 1364325000000 (전기), 통화: KRW',
 '순확정급여자산 : 3745697000000 (당기), 4410223000000 (전기), 통화: KRW'